In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states

# Census API Key
from config import api_key
c = Census(api_key, year=2013)

In [2]:
farm_markets = pd.read_csv("farmers_market.csv")

farm_markets.columns

Index(['FMID', 'MarketName', 'Website', 'Facebook', 'Twitter', 'Youtube',
       'OtherMedia', 'street', 'city', 'County', 'State', 'zip', 'Season1Date',
       'Season1Time', 'Season2Date', 'Season2Time', 'Season3Date',
       'Season3Time', 'Season4Date', 'Season4Time', 'x', 'y', 'Location',
       'Credit', 'WIC', 'WICcash', 'SFMNP', 'SNAP', 'Organic', 'Bakedgoods',
       'Cheese', 'Crafts', 'Flowers', 'Eggs', 'Seafood', 'Herbs', 'Vegetables',
       'Honey', 'Jams', 'Maple', 'Meat', 'Nursery', 'Nuts', 'Plants',
       'Poultry', 'Prepared', 'Soap', 'Trees', 'Wine', 'Coffee', 'Beans',
       'Fruits', 'Grains', 'Juices', 'Mushrooms', 'PetFood', 'Tofu',
       'WildHarvested', 'updateTime'],
      dtype='object')

In [3]:
farm_markets.head()

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,...,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,NaN,NaN,NaN,NaN,Danville,Caledonia,...,Y,Y,Y,N,N,Y,Y,N,N,6/20/17 22:43
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,NaN,NaN,NaN,6975 Ridge Road,Parma,Cuyahoga,...,N,N,Y,N,N,N,N,N,N,6/21/17 17:15
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,NaN,NaN,NaN,106 S. Main Street,Six Mile,NaN,...,N,N,N,N,N,N,N,N,N,2013
3,1010691,10th Steet Community Farmers Market,NaN,NaN,NaN,NaN,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,...,N,N,Y,N,N,N,N,N,N,10/28/14 9:49
4,1002454,112st Madison Avenue,NaN,NaN,NaN,NaN,NaN,112th Madison Avenue,New York,New York,...,N,N,N,N,N,N,N,N,N,3/1/12 10:38


In [4]:
farm_markets_clean = farm_markets.drop(["FMID", "Website", "Facebook", "Twitter", "Youtube",
                                       "OtherMedia", "street", "updateTime"], axis=1)
farm_markets_clean.head()

,MarketName,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,...,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested
0,Caledonia Farmers Market Association - Danville,Danville,Caledonia,Vermont,5828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,NaN,...,N,Y,Y,Y,N,N,Y,Y,N,N
1,Stearns Homestead Farmers' Market,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,NaN,NaN,NaN,...,N,N,N,Y,N,N,N,N,N,N
2,106 S. Main Street Farmers Market,Six Mile,NaN,South Carolina,29682,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
3,10th Steet Community Farmers Market,Lamar,Barton,Missouri,64759,04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,NaN,NaN,NaN,...,N,N,N,Y,N,N,N,N,N,N
4,112st Madison Avenue,New York,New York,New York,10029,July to November,Tue:8:00 am - 5:00 pm;Sat:8:00 am - 8:00 pm;,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N


In [5]:
farm_markets_clean.columns

Index(['MarketName', 'city', 'County', 'State', 'zip', 'Season1Date',
       'Season1Time', 'Season2Date', 'Season2Time', 'Season3Date',
       'Season3Time', 'Season4Date', 'Season4Time', 'x', 'y', 'Location',
       'Credit', 'WIC', 'WICcash', 'SFMNP', 'SNAP', 'Organic', 'Bakedgoods',
       'Cheese', 'Crafts', 'Flowers', 'Eggs', 'Seafood', 'Herbs', 'Vegetables',
       'Honey', 'Jams', 'Maple', 'Meat', 'Nursery', 'Nuts', 'Plants',
       'Poultry', 'Prepared', 'Soap', 'Trees', 'Wine', 'Coffee', 'Beans',
       'Fruits', 'Grains', 'Juices', 'Mushrooms', 'PetFood', 'Tofu',
       'WildHarvested'],
      dtype='object')

In [6]:
farm_markets_clean.count()

MarketName       8730
city             8690
County           8217
State            8730
zip              7793
Season1Date      5570
Season1Time      5733
Season2Date       455
Season2Time       444
Season3Date        82
Season3Time        79
Season4Date         6
Season4Time         6
x                8701
y                8701
Location         2587
Credit           8730
WIC              8730
WICcash          8730
SFMNP            8730
SNAP             8730
Organic          8730
Bakedgoods       8730
Cheese           8730
Crafts           8730
Flowers          8730
Eggs             8730
Seafood          8730
Herbs            8730
Vegetables       8730
Honey            8730
Jams             8730
Maple            8730
Meat             8730
Nursery          8730
Nuts             8730
Plants           8730
Poultry          8730
Prepared         8730
Soap             8730
Trees            8730
Wine             8730
Coffee           8730
Beans            8730
Fruits           8730
Grains    

In [7]:
farm_markets_df1 = farm_markets_clean.drop_duplicates(subset=['MarketName', 'city', 'County', 'State', 'zip'], keep='first')

In [8]:
farm_markets_df1.count()

MarketName       8687
city             8647
County           8174
State            8687
zip              7752
Season1Date      5530
Season1Time      5693
Season2Date       453
Season2Time       442
Season3Date        82
Season3Time        79
Season4Date         6
Season4Time         6
x                8658
y                8658
Location         2559
Credit           8687
WIC              8687
WICcash          8687
SFMNP            8687
SNAP             8687
Organic          8687
Bakedgoods       8687
Cheese           8687
Crafts           8687
Flowers          8687
Eggs             8687
Seafood          8687
Herbs            8687
Vegetables       8687
Honey            8687
Jams             8687
Maple            8687
Meat             8687
Nursery          8687
Nuts             8687
Plants           8687
Poultry          8687
Prepared         8687
Soap             8687
Trees            8687
Wine             8687
Coffee           8687
Beans            8687
Fruits           8687
Grains    

In [9]:
farm_markets_df2 = farm_markets_clean.drop_duplicates(subset=['MarketName', 'State', 'zip'], keep='first')

In [10]:
farm_markets_df2.count()

MarketName       8655
city             8615
County           8142
State            8655
zip              7725
Season1Date      5509
Season1Time      5666
Season2Date       451
Season2Time       440
Season3Date        81
Season3Time        78
Season4Date         6
Season4Time         6
x                8626
y                8626
Location         2547
Credit           8655
WIC              8655
WICcash          8655
SFMNP            8655
SNAP             8655
Organic          8655
Bakedgoods       8655
Cheese           8655
Crafts           8655
Flowers          8655
Eggs             8655
Seafood          8655
Herbs            8655
Vegetables       8655
Honey            8655
Jams             8655
Maple            8655
Meat             8655
Nursery          8655
Nuts             8655
Plants           8655
Poultry          8655
Prepared         8655
Soap             8655
Trees            8655
Wine             8655
Coffee           8655
Beans            8655
Fruits           8655
Grains    

In [11]:
farm_markets_df3 = farm_markets_clean.drop_duplicates(subset=['MarketName', 'zip'], keep='first')

In [12]:
farm_markets_df3.count()

MarketName       8653
city             8613
County           8140
State            8653
zip              7725
Season1Date      5508
Season1Time      5665
Season2Date       450
Season2Time       439
Season3Date        81
Season3Time        78
Season4Date         6
Season4Time         6
x                8624
y                8624
Location         2547
Credit           8653
WIC              8653
WICcash          8653
SFMNP            8653
SNAP             8653
Organic          8653
Bakedgoods       8653
Cheese           8653
Crafts           8653
Flowers          8653
Eggs             8653
Seafood          8653
Herbs            8653
Vegetables       8653
Honey            8653
Jams             8653
Maple            8653
Meat             8653
Nursery          8653
Nuts             8653
Plants           8653
Poultry          8653
Prepared         8653
Soap             8653
Trees            8653
Wine             8653
Coffee           8653
Beans            8653
Fruits           8653
Grains    

In [13]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)